# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [4]:
# number of callbacks for white-sounding names
sum(data[data.race=='w'].call)

235.0

In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

<b>#1. What test is appropriate for this problem? Does CLT apply?</b>

Of the 5000 resumes sent out for jobs, we want to know whether the proportion of successful resumes with black-sounding names is greater than or equal to the proportion of successful resumes with white-sounding names. To test this, we want a <b>two-sample t-test for a proportion.</b> 

<i>Note:</i> The website on the data states that 5000 resumes were sent out. The dataframe here, however, only has 4870 observations. I'll use n=4870 for this assignment.

<b>The Central Limit Theorem</b>--which posits that whatever distribution we start with, the frequency distribution would still be approximately normal over time--applies here. As the size of the sample increases, the more the frequency distribution of the data will approximate the normal distribution, and with a sample of 5000 resumes, we have enough data to do this.

<b>#2. What are the null and alternate hypotheses?</b>

H0 (null hypothesis): The proportion of successful resumes with black-sounding names is equal to the proportion of successful resumes with white-sounding names.

H1 (alternative hypothesis): The proportion of successful resumes with black-sounding names is not equal to the proportion of successful resumes with white-sounding names.

<div class="span5 alert alert-success">
<p>Your answer to Q3 here</p>
</div>

<b>#3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.</b>

In [6]:
w = data[data.race=='w']
b = data[data.race=='b']

Bootstrapping approach

In [7]:
def permutation_sample(data1, data2):
    """Generate a permutation sample from two data sets."""

    # Concatenate the data sets: data
    data = np.concatenate((data1, data2))

    # Permute the concatenated array: permuted_data
    permuted_data = np.random.permutation(data)

    # Split the permuted array into two: perm_sample_1, perm_sample_2
    perm_sample_1 = permuted_data[:len(data1)]
    perm_sample_2 = permuted_data[len(data1):]

    return perm_sample_1, perm_sample_2

In [8]:
def draw_perm_reps(data_1, data_2, func, size=1):
    """Generate multiple permutation replicates."""

    # Initialize array of replicates: perm_replicates
    perm_replicates = np.empty(size)

    for i in range(size):
        # Generate permutation sample
        perm_sample_1, perm_sample_2 = permutation_sample(data_1, data_2)

        # Compute the test statistic
        perm_replicates[i] = func(perm_sample_1, perm_sample_2)

    return perm_replicates

In [9]:
# Construct arrays of data: whites, blacks
whites = np.array([True] * 253 + [False] * 4617)
blacks = np.array([True] * 157 + [False] * 4713)

def frac_resume_success(blacks, reps):
    """Compute fraction of successful resume solicitations."""
    frac = np.sum(blacks) / len(blacks)
    return frac

# Acquire permutation samples: perm_replicates
perm_replicates = draw_perm_reps(whites, blacks, frac_resume_success, 10000)

# Compute 95% confidence interval: CI
CI = np.percentile(perm_replicates, [2.5, 97.5])

# Print the results
print('95% confidence interval =', CI)

# Compute and print p-value: p
p = np.sum(perm_replicates <= 157/4870) / len(perm_replicates)
print('p-value =', p)


95% confidence interval = [0.03819302 0.04599589]
p-value = 0.0


Frequentist approach

In [10]:
# Conduct t-test
stats.ttest_ind(data[data.race=='w'].call, data[data.race=='b'].call, equal_var = False)

Ttest_indResult(statistic=4.114705290861751, pvalue=3.942941513645935e-05)

<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

<b>#4. Write a story describing the statistical significance in the context of the original problem.</b>

The original problem was to "establish whether race has a significant impact on the rate of callbacks for resumes." Based on the statistical tests performed here, we can conclude with 95% confidence that the frequency distribution of black callbacks is not equal to the frequency distribution of white callbacks. Or, put in other words, blacks had a significantly lower frequency of callbacks within this dataset than whites. 

<b>#5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?</b>

No; if you look at the information provided alongside this dataset, there are other factors documented. While we can say with a certain level of confidence that race/name is an influential factor on callback success, there are potentially other significant factors that we haven't explored here. For example, we know that resumes were also sent out based on quality (with a high- and a low-quality resume sent out for each name). That would be a meaningful next step in determining what factors, generally, impact callback rate. In sum, while race/name may be a significant factor, we cannot make any other claims about other factors as we have not tested them.